In [1]:
import os
from os import listdir
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Get the parent directory
os.chdir(parent_dir)  # Change the working directory to the parent directory
os.sys.path.append(parent_dir)  # Append the parent directory to the system path
listdir()

['.DS_Store',
 'LICENSE',
 'util.py',
 'clustering',
 'preprocess',
 'model_manager.py',
 'README.md',
 'RAPTOR_util.py',
 '.gitignore',
 'postprocess',
 '.git',
 'summarizer',
 '.idea']

In [2]:
from preprocess.text_embedding import bcbert_encoder

In [3]:
t = bcbert_encoder("Medications")

In [4]:
import pandas as pd 
import numpy as np 

In [5]:
df = pd.read_csv("preprocess/train_fix_role_output_file.csv")

In [6]:
df 

,dataset,encounter_id,dialogue,note,clean_dialogue,restore_punctuation_dialogue,fixed_role_dialogue
0,virtassist,D2N001,"[doctor] hi , martha . how are you ?\n[patient...",CHIEF COMPLAINT\n\nAnnual exam.\n\nHISTORY OF ...,"[doctor] hi , martha . how are you ?\n[patient...","[doctor] hi martha, how are you?\n[patient] th...","[doctor] hi martha, how are you?\n[patient] th..."
1,virtassist,D2N002,"[doctor] hi , andrew , how are you ?\n[patient...",CHIEF COMPLAINT\n\nJoint pain.\n\nHISTORY OF P...,"[doctor] hi , andrew , how are you ?\n[patient...","[doctor] hi andrew, how are you?\n[patient] hi...","[doctor] hi andrew, how are you?\n[patient] hi..."
2,virtassist,D2N003,"[doctor] hi , john . how are you ?\n[patient] ...",CHIEF COMPLAINT\n\nBack pain.\n\nHISTORY OF PR...,"[doctor] hi , john . how are you ?\n[patient] ...","[doctor] hi john, how are you?\n[patient] hey,...","[doctor] hi john, how are you?\n[patient] hey,..."
3,virtassist,D2N004,"[doctor] hi , james , how are you ?\n[patient]...",CHIEF COMPLAINT\n\nBack pain.\n\nHISTORY OF PR...,"[doctor] hi , james , how are you ?\n[patient]...","[doctor] hi james, how are you?\n[patient] hey...","[doctor] hi james, how are you?\n[patient] hey..."
4,virtassist,D2N005,"[doctor] hey , ms. hill . nice to see you .\n[...",CC:\n\nRight middle finger pain.\n\nHPI:\n\nMs...,"[doctor] hey , ms. hill . nice to see you .\n[...","[doctor] hey, ms hill, nice to see you.\n[pati...","[doctor] hey, ms hill, nice to see you.\n[pati..."
...,...,...,...,...,...,...,...
62,aci,D2N063,[doctor] so gloria is a 46 -year-old female to...,CHIEF COMPLAINT\n\nDyspnea.\n\nMEDICAL HISTORY...,[doctor] so gloria is a 46 -year-old female to...,[doctor] so gloria is a 46 -year-old female to...,[doctor] so gloria is a 46 -year-old female to...
63,aci,D2N064,[doctor] hey matthew how're you doing\n[patien...,CHIEF COMPLAINT\n\nLeft ankle pain.\n\nHISTORY...,[doctor] hey matthew how're you doing\n[patien...,"[doctor] hey, matthew, how're you doing?\n[pat...","[doctor] hey, matthew, how're you doing?\n[pat..."
64,aci,D2N065,[doctor] hey anna good to see you today so i'm...,CHIEF COMPLAINT\n\nRight ankle pain.\n\nHISTOR...,[doctor] hey anna good to see you today so i a...,"[doctor] hey, anna, good to see you today. so ...","[doctor] hey, anna, good to see you today. so ..."
65,aci,D2N066,[doctor] hey gabriel i'm doctor scott good to ...,CHIEF COMPLAINT\n\nBack pain evaluation.\n\nME...,[doctor] hey gabriel i am doctor scott good to...,"[doctor] hey, gabriel, i am doctor scott. good...","[doctor] hey, gabriel, i am doctor scott. good..."


In [7]:
utt = df.fixed_role_dialogue[0].split("\n")

In [8]:
print(utt)

['[doctor] hi martha, how are you?', '[patient] the patient is doing okay. how are you?', "[doctor] i am doing okay. so i know the nurse told you about dax. i'd to tell dax a little bit about you, okay.", '[patient] okay.', '[doctor] martha is a 50-year-old female with a past medical history significant for congestive heart failure, depression and hyptension, who presents for h annual exam. so, martha, it is been a year since i have seen you. how are you doing?', '[patient] the patient is doing. the patient has been traveling a lot recently since things have gotten a bit light and the patient got his / her vaccine. so the patient feel saf about traveling. the patient has been doing a lot of hiking. went to washington last weekend to hike in northn cascades around the mount bak area.', "[doctor] nice, that's great. i am glad to hear that you are staying active. i just love this weath. i am so happy the sm is ov. i am definitely more of a fall pson.", '[patient] yes, fall foliage is the 

In [9]:
t2 = bcbert_encoder(utt[0])

In [11]:
def cossim(A,B) : 
    return np.dot(A,B).sum() / (np.sqrt((A**2).sum()) * np.sqrt((B**2).sum()))

In [12]:
cossim(t[0],t2[0])

tensor(0.5855)

In [13]:
df2 = pd.read_csv("preprocess/processed_test.csv")

In [14]:
col = df2.columns

In [15]:
col 

Index(['id', 'text', 'CHIEF_COMPLAINT', 'HISTORY_OF_PRESENT_ILLNESS',
       'Medications', 'Surgical', 'CURRENT_MEDICATIONS', 'Weight', 'Height',
       'Blood_pressure_BP', 'Respirations_Respiratory_rate_RR',
       'Pulse_oxygenation/_SpO2/Oxygen_saturation', 'Temperature',
       'Heart_rate_HR/_Pulse', 'MEDICATIONS', 'ALLERGIES', 'SOCIAL_HISTORY',
       'MEDICAL_HISTORY', 'FAMILY_HISTORY', 'PAST_SURGICAL_HISTORY',
       'Psychiatric', 'Neurologic_NEURO', 'Neck',
       'Head,_Ears,_Nose,_Mouth_and_Throat', 'Musculoskeletal_MSK',
       'Gastrointestinal', 'Genitourinary', 'Constitutional', 'Respiratory',
       'Cardiovascular_CV', 'Blood_Pressure', 'RESULTS', 'ASSESSMENT_AND_PLAN',
       'Breast', 'Strength', 'Sensation', 'Skin', 'Physical', 'Eyes',
       'Hematology/Lymphatic/Immunology', 'Pelvic', 'Rectal', 'Integumentary'],
      dtype='object')

In [16]:
point = [col[i] for i in range(len(col)) if not col[i].isupper() and col[i] not in ['id','text']]

In [17]:
l = [cossim(bcbert_encoder(point[i]),t2[0]) for i in range(len(point))]

In [18]:
np.array(l)

array([0.5854563 , 0.59730524, 0.54706615, 0.5880053 , 0.7537563 ,
       0.7717687 , 0.77129567, 0.6002998 , 0.7639954 , 0.6312064 ,
       0.77058965, 0.61408955, 0.7955163 , 0.8049327 , 0.6831906 ,
       0.74062467, 0.6061513 , 0.6015893 , 0.78652483, 0.7183288 ,
       0.60099983, 0.5789773 , 0.6165299 , 0.6324486 , 0.606112  ,
       0.6244447 , 0.7525525 , 0.6873409 , 0.7021069 , 0.73557425],
      dtype=float32)

In [19]:
point

['Medications',
 'Surgical',
 'Weight',
 'Height',
 'Blood_pressure_BP',
 'Respirations_Respiratory_rate_RR',
 'Pulse_oxygenation/_SpO2/Oxygen_saturation',
 'Temperature',
 'Heart_rate_HR/_Pulse',
 'Psychiatric',
 'Neurologic_NEURO',
 'Neck',
 'Head,_Ears,_Nose,_Mouth_and_Throat',
 'Musculoskeletal_MSK',
 'Gastrointestinal',
 'Genitourinary',
 'Constitutional',
 'Respiratory',
 'Cardiovascular_CV',
 'Blood_Pressure',
 'Breast',
 'Strength',
 'Sensation',
 'Skin',
 'Physical',
 'Eyes',
 'Hematology/Lymphatic/Immunology',
 'Pelvic',
 'Rectal',
 'Integumentary']

In [20]:
def embed(point, Utt) : 
    embedded_pt = bcbert_encoder(point)[0]
    embedded_Utt = bcbert_encoder(Utt)[0]
    return embedded_pt, embedded_Utt

In [21]:

class Node:
    """
    Represents a node in the hierarchical tree structure.
    """

    def __init__(
            self, text: str, index: int, embeddings
    ) -> None:
        self.text = text
        self.index = index
        self.embeddings = embeddings


class Tree:
    """
    Represents the entire hierarchical tree structure.
    """

    def __init__(
            self, all_nodes, root_nodes, leaf_nodes, num_layers, layer_to_nodes
    ) -> None:
        self.all_nodes = all_nodes
        self.root_nodes = root_nodes
        self.leaf_nodes = leaf_nodes
        self.num_layers = num_layers
        self.layer_to_nodes = layer_to_nodes


In [22]:
def create_node(
        file_path: str, 
        dialogue_column: str, 
        task='C', 
        exceptions = None
):
    """
    Create a list of lists of Nodes, where each list represents all Nodes of a single sample.

    Args:
        file_path (str): Path to the CSV file containing dialogue data.
        dialogue_column (str): The name of the column in the DataFrame that contains the dialogue text.
        task (str): A task identifier; default is 'C'.
        exceptions (List[List[str]]): Optional list of [old_text, new_text] pairs for text replacements.

    Returns:
        List[List[Node]]: A nested list where each inner list contains Node objects for a single sample.
    """
    # Load CSV file into a DataFrame
    df = pd.read_csv(file_path)

    results = []

    for i, row in df.iterrows():
        print(i)
        # Extract and clean text from the specified dialogue column
        text = row[dialogue_column].replace('\r', '')

        # Apply exception replacements, if provided
        if exceptions:
            for old_text, new_text in exceptions:
                text = text.replace(old_text, new_text)

        # Split the processed text into utterances based on newlines
        utterances = text.split('\n')

        # Convert each utterance into a Node with its corresponding embedding
        nodes = [
            Node(text=utter, index=i, embeddings=bcbert_encoder(utter))
            for i, utter in enumerate(utterances)
        ]

        # Append the list of Nodes for the current row to the results
        results.append(nodes)

    return results




In [42]:
l = np.array([1,2,4,5,6,7,4,5])
np.argsort(l)[::-1]

l

array([5, 4, 7, 3, 6, 2, 1, 0])

In [40]:
l

In [23]:
def sort_section(list_section, utt, thres) : 

    for i in range(len(list_section)) : 
        embedded_pt, embedded_Utt = embed(list_section[i],utt)
        l = [cossim(embedded_pt,embedded_Utt) for i in range(len(list_section))]
    
    

In [27]:
utt = utt[:10]

In [28]:
nodes = []
for i, utter in enumerate(utt) : 
    print(i)
    node = Node(text=utter, index=i, embeddings=bcbert_encoder(utter))
    nodes.append(node)

0
1
2
3
4
5
6
7
8
9


In [30]:
from clustering.tree_builder import create_node

In [ ]:
results = [[] for _ in range(len(col))]

for i in range(len(nodes)) : 
    l = np.array([cossim(bcbert_encoder(col[j])[0],nodes[i].embeddings[0]) for j in range(len(col))])

    index = [i for i in range(len(l)) if l[i] >= 0.5]
    for j in range(len(index)) : 
        results[j].append(nodes[i])